<a href="https://colab.research.google.com/github/Elmermd/Academia-IA/blob/main/Actividad_EjercicioFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inteligencia Artificial y Aprendizaje Automático**
# Semillero IA & Data
## Actividad Ejercicio Final : – ML regresión logística y Métricas de desempeño- Riesgo Crediticio

###Alumno: **Elías Basaldúa García**

In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

import zipfile
import io
import requests

# Carga los datos

In [33]:
# Archivo ZIP
url = "https://archive.ics.uci.edu/static/public/522/south+german+credit.zip"

response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
  # Leer el archivo
  with z.open("SouthGermanCredit.asc") as f:
      df = pd.read_csv(f, delim_whitespace=True, header=None)

<ipython-input-33-c20313575f2f>:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(f, delim_whitespace=True, header=None)


# Sustituye los nombres de las columnas del alemán al inglés de acuerdo con como se indica en la página de la UCI.

In [26]:
df = df.drop(index=0).reset_index(drop=True)
df.columns = [
    'status', 'duration', 'credit_history', 'purpose', 'amount',
    'savings', 'employment', 'installment_rate', 'personal_status_sex',
    'other_debtors', 'present_residence', 'property', 'age',
    'other_installment_plans', 'housing', 'number_credits',
    'job', 'people_liable', 'telephone', 'foreign_worker', 'credit_risk'
]

df.head()

,status,duration,credit_history,purpose,amount,savings,employment,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,credit_risk
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


# Realiza una partición de los datos en el conjunto de entrenamiento del 85% y el de prueba de 15%. Los modelos se estarán entrenando con el método de validación cruzada, así que no es necesario en este paso generar el conjunto de validación. Define como la variable X a todas las variables de entrada y a la variable Y como la variable de salida.

In [27]:
# Definimos nuestros datos de entrada, de salida y la partición correspondiente:
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]

Xtv, Xtest, ytv, ytest = train_test_split(X, Y, test_size=0.15, random_state=1, stratify=Y)

print(Xtv.shape, ': dimensión de datos de entrada de entrenamiento y validación')
print(Xtest.shape, ': dimensión de datos de entrada de prueba')
print(ytv.shape, ': dimensión de variable de salida para entrenamiento y validación')
print(ytest.shape, ': dimensión de variable de salida para prueba')

(850, 20) : dimensión de datos de entrada de entrenamiento y validación
(150, 20) : dimensión de datos de entrada de prueba
(850,) : dimensión de variable de salida para entrenamiento y validación
(150,) : dimensión de variable de salida para prueba


**Como una primera aproximación (baseline) realizarás las siguientes transformaciones mínimas para generar los primeros modelos. En la misma página de la UCI se indica el tipo de variable de cada uno de los factores. Definen cuatro tipos de variables: categórica (categorical), ordinal (discretized quantitative), numérica (quantitative) y binaria (binary). Con base a dicha información realiza un Pipeline que incluya al menos las siguientes transformaciones:**
- a. Imputación a todas las variables de entrada, diferenciando entre el tipo de cada variable (decide y justifica que tipo de imputación realizas en cada caso).
- b. Realiza un análisis de las variables numéricas (quantitative) de entrada y aplica una transformación que escale a todas ellas en un rango equiparable.
- c. Aplica la transformación One-Hot encoding a las variables de entrada de tipo categórico y binaria. En particular, justifica por qué una variable binaria requeriría que se le aplique la transformación one-hot encoding. Por el momento dejar las variables ordinales sin transformar.

In [28]:
Xtv = Xtv.copy()
Xtv.columns = [
    'A1', 'A2', 'A3', 'A4', 'A5',
    'A6', 'A7', 'A8', 'A9', 'A10',
    'A11', 'A12', 'A13', 'A14', 'A15',
    'A16', 'A17', 'A18', 'A19', 'A20'
]
Xtv

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
655,2,18,3,9,2427,5,5,4,3,1,2,2,42,3,2,2,3,2,1,2
843,2,24,2,10,11328,1,3,2,3,2,3,3,29,1,2,2,4,2,2,2
221,3,30,4,3,3656,5,5,4,3,1,4,2,49,2,2,2,2,2,1,2
825,2,6,3,0,1209,1,1,4,3,1,4,2,47,3,2,1,4,2,2,2
60,4,12,4,0,1163,3,3,4,3,1,4,1,44,3,2,1,3,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,2,12,2,3,766,3,3,4,3,1,3,1,66,3,2,1,2,2,1,2
92,2,30,0,9,4221,1,3,2,2,1,1,3,28,3,2,2,3,2,1,2
761,1,18,2,3,2600,1,3,4,3,1,4,4,65,3,3,2,3,2,1,2
233,1,30,2,1,3857,1,3,4,1,1,4,2,40,3,2,1,4,2,2,2


In [29]:
# Variables: A2 = duration, A3 = amount, A13 = age
num_pipeline = Pipeline(steps=[
    ('impMediana', SimpleImputer(strategy='median')),
    ('escalaNum', MinMaxScaler(feature_range=(1, 2)))
])
num_pipeline_nombres = ['A2', 'A3', 'A13']

In [30]:
catImp_pipeline = Pipeline(steps=[
    ('impModa', SimpleImputer(strategy='most_frequent'))
])
catImp_pipeline_nombres = ['A1', 'A8', 'A9', 'A11', 'A14', 'A15', 'A18']

In [31]:
catOHE_pipeline = Pipeline(steps=[
    ('OneHotE', OneHotEncoder(drop='first'))
])
catOHE_pipeline_nombres = ['A4', 'A5', 'A6', 'A12', 'A19', 'A20']

In [32]:
columnasTransformer = ColumnTransformer(transformers=[
    ('numpipe', num_pipeline, num_pipeline_nombres),
    ('catimp', catImp_pipeline, catImp_pipeline_nombres),
    ('catohe', catOHE_pipeline, catOHE_pipeline_nombres)
], remainder='passthrough')

4.- Llevarás a cabo un entrenamiento utilizando validación cruzada exclusivamente con el modelo de Regresión Logística. Deberás realizar el entrenamiento aplicando las transformaciones que definiste en tu Pipeline. El entrenamiento debe cumplir con las siguientes características:
- a. Usa los parámetros predeterminados del modelo de Regresión Logística.
- b. En cada iteración deberán calcularse las siguientes métricas: accuracy, precision, recall, f1-score y Gmean. Todas estas métricas deben ser funciones que tú mismo definas (es decir, no usar las funciones ya integradas de scikit-learn, aunque sí puedes utilizar la información proporcionada por el método confusion_matrix() para definirlas).
- c. Utiliza validación cruzada estratificada con 5 particiones y 3 repeticiones.
- d. Imprime el valor de todas estas métricas, tanto para los datos de entrenamiento como para los de validación. Además, realiza diagramas de caja y bigotes (boxplots) con la métrica recall para visualizar los resultados de las repeticiones.
- e. Reflexiona: ¿El modelo presenta indicios de estar subentrenado o sobreentrenado? Justifica tu
respuesta.